# Lab 3: Key Exchange and Breaking DES
## 1. Breaking DES
The key space of DES is $2^{56}$. If a secret key `k` was selected purely at random from the keyspace, an adversary would on average have to attempt $2^{55}$ possible keys (half the key space) before encountering the correct key.
### 1.1 Questions
Imagine you were performing a chosen-plaintext attack on DES. Is there a property of DES that would allow you to reduce your work by a factor of 2 (i.e. to $2^{54}$)?

DESX is an improvement on the strength of normal DES by adding two extra keys, `DESX(m) = k3 ⊕ Ek1(m ⊕ k2)`. Imagine we created DES$X/2$ (or
DES Half X) by instead computing `E(m) = k3 ⊕ Ek1(m)`. How would you go about breaking this cypher, assuming you have two plaintext-cyphertext pairs. Thus, how much stronger is DES$X/2$ compared to normal DES? Why does the whitening key added to DESX (i.e. `m ⊕ k2`) prevent the the attack we used on DES$X/2$?

## 2 Merkle’s Puzzles
Merkle’s Puzzles is a conceptually simple public-key cryptosystem. Alice, wanting to speak securely to Bob, generates `m` boxes and sends them to Bob. Each box is encrypted using an easily broken cypher (for some definition of “easily”) and contains the box number `bi` and a stronger shared secret `k`.

Bob selects a box at random, breaks it, and sends the previously secret box number `bi` back to Alice. Alice and Bob now have a shared secret `k`.

To retrieve their shared secret, Bob only needs to open a single box. Our eavesdropping attacker Eve however would statistically need to open $m/2$ boxes before finding box `bi`.

Attached to this lab is the source code for a minimal Merkle Puzzle system. For each of the puzzles, a key and the box number are encrypted using a simple DES cypher. Note the speed at which a large number of puzzles may be generated and how slow it can be to break each of these boxes. The latter can be seen by increasing or decreasing the key complexity.


In [1]:
# Merkle.java from lab converted to Python

import string
import random
import time
from Crypto.Cipher import DES


# Make a random string length l
def random_bytes(l):
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k = l)).encode()


# Make a random DES-compatible key length l
def  random_key(l):
    
    k = random_bytes(l) + b"00000000" 
    return k[0:8] 


# Encrypt with DES
def  encrypt(k,  m):
    
    cipher = DES.new(k, DES.MODE_ECB)
    while len(m) % 8: 
        m = m + b" " # Pad message with whitespace for ECB mode
    c = cipher.encrypt(m)
    return c


# Decrypt with DES
def  decrypt(k,  c) :
 
    cipher = DES.new(k, DES.MODE_ECB)
    m = cipher.decrypt(c)
    m = m.rstrip() # Remove padding
    return m


key_len = 4 # Values greater than 8 will be reduced to 8 by the random_key function, as required by DES
n_puzzles = 10000
print(f"Alice and Bob agree the puzzle key strings will be padded from length %d, and that Alice will send %d puzzles.\n" % (key_len, n_puzzles))

puzzles =  []
for i in range(n_puzzles):
    k = random_key(key_len)
    m = b"Key=" + random_bytes(30) + b" & Puzzle=" + str(i).encode()
    c = encrypt(k, m)
    puzzles.append(c)
print(f"Alice generates the %d puzzles. She records their order and their contents.\n" % n_puzzles)
print(f"Alice encrypts all of the puzzles using DES and shuffles them around so they are no longer in order.\n")
shuffled = random.sample(puzzles, n_puzzles)

print("Alice sends the shuffled, encrypted collection of puzzles to Bob.\n")

i = random.randint(0, n_puzzles)
c = shuffled[i]
print(f"Bob randomly selects the puzzle at position %d in the shuffled list. It looks like the ciphertext %s.\n" % (i, c))

print("Bob now attempts to brute-force the known-length key to his chosen puzzle.\n")
tic = time.time()
m = None
attempts = 0
while (m is None or not b"Key=" in m):
    
    attempts += 1
    k = random_key(key_len)
    m = decrypt(k, c)
toc = time.time() - tic

print("Bob guesses the key in %d attempts over approximately %d seconds and retrieves the message: %s.\n" % (attempts, toc, m))

i = int(m[44:].decode())
print(f"Bob sends back to Alice that he opened puzzle %d.\n" % i)

print(f"Alice looks at her original non-shuffled, unencrypted list of puzzles and picks number %d. She now has the same key as Bob.\n" % i)
print(f"Eve, who was watching communications both ways, cannot know the key without opening every puzzle, which would take her on average %d times as long as it took Bob to open one puzzle, or approximately %d seconds.\n" % ((n_puzzles/2), (n_puzzles*toc/2)))


Alice and Bob agree the puzzle key strings will be padded from length 4, and that Alice will send 10000 puzzles.

Alice generates the 10000 puzzles. She records their order and their contents.

Alice encrypts all of the puzzles using DES and shuffles them around so they are no longer in order.

Alice sends the shuffled, encrypted collection of puzzles to Bob.

Bob randomly selects the puzzle at position 5104 in the shuffled list. It looks like the ciphertext b'\xa9a\xf8\xef\xff\x19$\xedG\xa5\xa2\x90\x01\x08\xf4t\x9dB\x8f\x80P\xc4\xde\xd5\xd3\x82\xca\xd6\xe3\xd8\xde\x08I\xd1\x8b\xd1\xfc\xb4\xa9\x00?\x8c\xb8\x86\xffs~\xad'.

Bob now attempts to brute-force the known-length key to his chosen puzzle.

Bob guesses the key in 34563 attempts over approximately 0 seconds and retrieves the message: b'Key=TIRWFENYVFXESB26SYD597Z6ZJSQOD & Puzzle=4883'.

Bob sends back to Alice that he opened puzzle 4883.

Alice looks at her original non-shuffled, unencrypted list of puzzles and picks number 4883.

### 2.1 Questions
In the minimal Merkle Puzzle system example above, a symmetric cypher (DES) is used to “secure” the puzzle. Can a puzzle be secured using a Hashbased Message Authentication Codes (HMAC)? If so, how so? If not, why not?


We have only considered an eavesdropping attacker Eve so far. Mallory, as oposed to Eve, can modify messages and even create her own. Does Merkle’s Puzzles by itself defend against Mallory in any way? Why not?

## 3. Diffie-Hellman Key Exchange

Ensure you understand the steps for the Diffie-Hellman key exchange, particularly which variables become public and which remain private. A reduced version is given below but refer to Wikipedia or the Handbook of Applied Cryptography for more details.

`A → B : α, p, α^a mod p`

`B → A : α^b mod p`

Shared secret `s = (α^a mod p) b mod p = (α^b mod p) a mod p`

Shared key = `hash(s)`
